## Importing Libraries


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
import random
import json
import keras
import pickle
from keras import optimizers
from keras.utils import np_utils
from numpy import argmax
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,BatchNormalization

Using TensorFlow backend.


## Building dataset

In [2]:
#creating array ranging from 1 to 211
CATEGORIES = []
for i in range (1,212):
    CATEGORIES.append(str(i))

In [3]:
IMG_SIZE = 75

In [4]:
DATADIR = "train"
training_data = []
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass

            
create_training_data()

In [5]:
# shuffle the features
random.shuffle(training_data)

In [6]:
# x and y array to store input and output
x = []
y = []

In [7]:
#filling of x and y with features and labels respectively
for features, label in training_data:
    x.append(features)
    y.append(label)
#reshaping of x    
x = np.array(x).reshape(-1,IMG_SIZE,IMG_SIZE,3)    

In [8]:

pickle_out = open("x.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

pickle_in = open("x.pickle","rb")
x = pickle.load(pickle_in)
x = pickle.load(open("x.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

In [9]:
#normalizing values between 0 to 1
x = x/255.0

In [10]:
#shaping y to work on category(211)
y = np_utils.to_categorical(y, 211)

# CNN MODEL

In [15]:
model = Sequential()


model.add(Conv2D(filters=32, 
               kernel_size=(2,2), 
               strides=(1,1),
               padding='same',
               input_shape=x.shape[1:],
               data_format='channels_last'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

"""model.add(Conv2D(filters=32,
               kernel_size=(2,2),
               strides=(1,1),
               padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
"""
model.add(Conv2D(filters=64,
               kernel_size=(2,2),
               strides=(1,1),
               padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
"""
model.add(Conv2D(filters=128,
               kernel_size=(2,2),
               strides=(1,1),
               padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),
                     strides=2))
"""

model.add(Flatten())        
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(211))

model.add(Activation('softmax'))

# Summary of model

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 75, 75, 32)        416       
_________________________________________________________________
activation_4 (Activation)    (None, 75, 75, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 36, 36, 64)        8256      
_________________________________________________________________
activation_5 (Activation)    (None, 36, 36, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)            

# Fitting of model

In [17]:
model.compile(loss = "categorical_crossentropy",optimizer = 'rmsprop' ,metrics = ['categorical_accuracy'])
model.fit(x,y,batch_size = 32,epochs = 10, validation_split = 0.2)

Train on 5636 samples, validate on 1410 samples
Epoch 1/10
5636/5636 [==============================] - 43s 8ms/sample - loss: 5.1796 - categorical_accuracy: 0.0270 - val_loss: 4.8617 - val_categorical_accuracy: 0.0716
Epoch 2/10
5636/5636 [==============================] - 41s 7ms/sample - loss: 4.3053 - categorical_accuracy: 0.1482 - val_loss: 4.2619 - val_categorical_accuracy: 0.1461
Epoch 3/10
5636/5636 [==============================] - 41s 7ms/sample - loss: 3.1521 - categorical_accuracy: 0.3419 - val_loss: 2.9632 - val_categorical_accuracy: 0.4007
Epoch 4/10
5636/5636 [==============================] - 41s 7ms/sample - loss: 2.2470 - categorical_accuracy: 0.5098 - val_loss: 2.3438 - val_categorical_accuracy: 0.5113
Epoch 5/10
5636/5636 [==============================] - 41s 7ms/sample - loss: 1.5191 - categorical_accuracy: 0.6469 - val_loss: 2.5287 - val_categorical_accuracy: 0.5355
Epoch 6/10
5636/5636 [==============================] - 42s 8ms/sample - loss: 1.0706 - categoric

In [18]:
# Reading country labels
with open("country.json", "r") as read_file:
    data = json.load(read_file)

In [19]:
#creating dictionary of country labels
CAT = {}
for i in range (1,212):
    CAT[str(i-1)]=data[str(i)].split(',')[2]

In [20]:
#Function to prepare test data compatible to feed into to model
def prepare(filepath):
    IMG_SIZE = 75
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,3)


In [36]:
#Preparation of test data set to feed into model and store results in results
path="test"
result=[]
for img in range (1,1056):     #os.listdir(path):
    #print (img)
    prediction = model.predict(prepare(os.path.join(path,str(img)+".jpg")))
    inverted = argmax(prediction[0])
    result.append(CAT[str(inverted)])

In [33]:
#Converting results to pandas dataframe to export it to json file
Y_test=pd.DataFrame(result)

In [ ]:
#Exporting result to csv file
Y_test.index+=1
Y_test.index
export_csv = Y_test.to_csv ('./Ytest.csv', index = True, header=True)

In [34]:
#Individual prediction
prediction = model.predict([prepare('./test/700.jpg')])
inverted = argmax(prediction[0])
print(CAT[str(inverted)])

india


In [38]:
#print (result)